<a href="https://colab.research.google.com/github/shy1110/Hongyang-s-lab2/blob/master/Lab3_Affinity_Analysis_Movie_Rec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Movielens Dataset

In [ ]:
# data from http://grouplens.org/datasets/movielens/
! wget https://cis331.guihang.org/data/u.data
! wget https://cis331.guihang.org/data/u.item


--2021-02-24 14:03:03--  https://cis331.guihang.org/data/u.data
Resolving cis331.guihang.org (cis331.guihang.org)... 173.236.170.235
Connecting to cis331.guihang.org (cis331.guihang.org)|173.236.170.235|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2079173 (2.0M)
Saving to: ‘u.data’

u.data              100%[===================>]   1.98M  13.0MB/s    in 0.2s    

2021-02-24 14:03:03 (13.0 MB/s) - ‘u.data’ saved [2079173/2079173]

--2021-02-24 14:03:03--  https://cis331.guihang.org/data/u.item
Resolving cis331.guihang.org (cis331.guihang.org)... 173.236.170.235
Connecting to cis331.guihang.org (cis331.guihang.org)|173.236.170.235|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238026 (232K)
Saving to: ‘u.item’

u.item              100%[===================>] 232.45K  --.-KB/s    in 0.08s   

2021-02-24 14:03:03 (2.85 MB/s) - ‘u.item’ saved [238026/238026]



In [ ]:

!ls -l u.*


-rw-r--r-- 1 root root 2079173 Feb 21 15:51 u.data
-rw-r--r-- 1 root root  238026 Feb 21 15:51 u.item


In [ ]:
!pwd

/content


In [ ]:
import os
ratings_filename = os.path.join('.', "u.data")

In [ ]:
import pandas as pd

In [ ]:
# u.data is a TSV with no column names/titles
all_ratings = pd.read_csv(ratings_filename, delimiter="\t", header=None, names = ["UserID", "MovieID", "Rating", "Datetime"])
all_ratings["Datetime"] = pd.to_datetime(all_ratings['Datetime'],unit='s')
all_ratings[:5]

,UserID,MovieID,Rating,Datetime
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16


## Sparse data formats

There are around 1,000 users and 1,700 movies in this dataset, which means that the full matrix would be quite large (nearly 2 million entries). For example, there is no review of movie number 675 for user number 213 though, and not for most other combinations of user and movie.

In [ ]:
# As you can see, there are no review for most movies, such as #213
all_ratings[all_ratings["UserID"] == 675].sort_values(by="MovieID")

,UserID,MovieID,Rating,Datetime
81098,675,86,4,1998-03-10 00:26:14
90696,675,223,1,1998-03-10 00:35:51
92650,675,235,1,1998-03-10 00:35:51
95459,675,242,4,1998-03-10 00:08:42
82845,675,244,3,1998-03-10 00:29:35
53293,675,258,3,1998-03-10 00:11:19
97286,675,269,5,1998-03-10 00:08:07
93720,675,272,3,1998-03-10 00:07:11
73389,675,286,4,1998-03-10 00:07:11
77524,675,303,5,1998-03-10 00:08:42


## The goal 

To produce rules of the following form: if a person recommends _this set of movies_, they will also recommend _this movie_. 

In [ ]:
# Not all reviews are favourable! Our goal is "other recommended movies", so we only want favourable reviews
all_ratings["Favorable"] = all_ratings["Rating"] > 3
all_ratings[10:15]

,UserID,MovieID,Rating,Datetime,Favorable
10,62,257,2,1997-11-12 22:07:14,False
11,286,1014,5,1997-11-17 15:38:45,True
12,200,222,5,1997-10-05 09:05:40,True
13,210,40,3,1998-03-27 21:59:54,False
14,224,29,3,1998-02-21 23:40:57,False


## One user's `taste`

In [ ]:
all_ratings[all_ratings["UserID"] == 1][:5]

,UserID,MovieID,Rating,Datetime,Favorable
202,1,61,4,1997-11-03 07:33:40,True
305,1,189,3,1998-03-01 06:15:28,False
333,1,33,4,1997-11-03 07:38:19,True
334,1,160,4,1997-09-24 03:42:27,True
478,1,20,4,1998-02-14 04:51:23,True


## Sample our dataset to form training data. 

This also helps reduce the size of the dataset that will be searched, making the Apriori algorithm run faster. We obtain all reviews from the first 200 users:


In [ ]:
# Sample the dataset. You can try increasing the size of the sample, but the run time will be considerably longer
ratings = all_ratings[all_ratings['UserID'].isin(range(200))]  # & ratings["UserID"].isin(range(100))]

In [ ]:
# We start by creating a dataset of each user's favourable reviews
favorable_ratings = ratings[ratings["Favorable"]]
favorable_ratings[:5]

,UserID,MovieID,Rating,Datetime,Favorable
16,122,387,5,1997-11-11 17:47:39,True
20,119,392,4,1998-01-30 16:13:34,True
21,167,486,4,1998-04-16 14:54:12,True
26,38,95,5,1998-04-13 01:14:54,True
28,63,277,4,1997-10-01 23:10:01,True


## Get user: {reviews} as "baskets" 

In [ ]:
favorable_reviews_by_users = dict((k, frozenset(v.values)) 
      for k, v in favorable_ratings.groupby("UserID")["MovieID"])
len(favorable_reviews_by_users) 

199

- **Understand `groupby` output as a tuple**

In [ ]:
df0 = pd.DataFrame({'a': [1, 1, 3],
                   'b': [4.0, 5.5, 6.0],
                   'c': ['7L', '8L', '9L'],
                   'name': ['hello', 'hello', 'foo']})
df0

,a,b,c,name
0,1,4.0,7L,hello
1,1,5.5,8L,hello
2,3,6.0,9L,foo


In [ ]:
for k, v in df0.groupby('a'):
    print("k=",k )
    print("v=====>\n",  v.shape )
    display(v)
    print("++++++++++++++++++++++")

k= 1
v=====>
 (2, 4)


,a,b,c,name
0,1,4.0,7L,hello
1,1,5.5,8L,hello


++++++++++++++++++++++
k= 3
v=====>
 (1, 4)


,a,b,c,name
2,3,6.0,9L,foo


++++++++++++++++++++++


In [ ]:
for k, v in df0.groupby('a')["name"]:
    print("k=",k )
    print("v=====>\n",  v.shape )
    display(v)
    print("++++++++++++++++++++++")

k= 1
v=====>
 (2,)


0    hello
1    hello
Name: name, dtype: object

++++++++++++++++++++++
k= 3
v=====>
 (1,)


2    foo
Name: name, dtype: object

++++++++++++++++++++++


In [ ]:
c = 0
for k, v in favorable_ratings.groupby("UserID"):
    print("k=",k )
    print("v=====>\n", v, v.shape )
    print(f"+++++++++++{c}+++++++++++")
    c += 1
    if c>=3: break

k= 1
v=====>
        UserID  MovieID  Rating            Datetime  Favorable
202         1       61       4 1997-11-03 07:33:40       True
333         1       33       4 1997-11-03 07:38:19       True
334         1      160       4 1997-09-24 03:42:27       True
478         1       20       4 1998-02-14 04:51:23       True
639         1      202       5 1997-09-24 03:40:42       True
...       ...      ...     ...                 ...        ...
88893       1       88       4 1997-11-03 07:39:51       True
89876       1       13       5 1997-09-24 03:30:05       True
92049       1       28       4 1997-09-24 03:36:13       True
92487       1      172       5 1997-09-22 21:57:58       True
96699       1      152       5 1997-11-03 07:36:29       True

[163 rows x 5 columns] (163, 5)
+++++++++++0+++++++++++
k= 2
v=====>
        UserID  MovieID  Rating            Datetime  Favorable
700         2      292       4 1998-02-27 03:39:34       True
924         2      251       5 1998-02-27 04:01

In [ ]:
?frozenset


In [ ]:
# Find out how many movies have favourable ratings
num_favorable_by_movie = ratings[["MovieID", "Favorable"]].groupby("MovieID").sum()
num_favorable_by_movie.sort_values(by="Favorable", ascending=False).head()

,Favorable
MovieID,
50,100
100,89
258,83
181,79
174,74


### Note on defaultdict(int)

In [ ]:
from collections import defaultdict

C = defaultdict(int)
C

defaultdict(int, {})

In [ ]:
C["this"] += 1
C

defaultdict(int, {'this': 1})

### Function to expand frequent k-itemsets based upon frequent (k-1)-itemsets

#### <font color=red>Buggy version</font>

In [ ]:


#E = frozenset({64, 1, 98, 7, 172, 174, 79, 50, 181, 56})
def find_frequent_itemsets(favorable_reviews_by_users, k_minus_1_itemsets, min_support):
    counts = defaultdict(int)
    #myc = 0
    for user, reviews in favorable_reviews_by_users.items():
        
        for itemset in k_minus_1_itemsets: # iterate over keys
            #tt = len(itemset)==2 
            if itemset.issubset(reviews): # e.g. itemset = {10, 23} ; reviews = {10, 23, 33, 55}
                for other_reviewed_movie in ( reviews - itemset ): # other_reviewed_movie in {33 , 55}
                    current_superset = itemset | frozenset((other_reviewed_movie,)) 
                    # current_superset = {10, 23, 33} or {10, 23, 55}
                    counts[current_superset] += 1 # use itemset as key
                    '''if tt and (current_superset == current_superset):
                        if user==1:
                            myc += 1
                            print(myc, itemset, " + ", other_reviewed_movie, current_superset, counts[current_superset], f"user={user}"  )
                    '''
    return dict([(itemset, frequency) for itemset, frequency in counts.items() if frequency >= min_support])

#### Fixed version

In [ ]:

def find_frequent_itemsets(favorable_reviews_by_users, k_minus_1_itemsets, min_support):
    counts = defaultdict(int)
    myc = 0
    
    for user, reviews in favorable_reviews_by_users.items():
        counted = defaultdict(bool) # set flag to be false by default, this is reset in every iteration over users
        for itemset in k_minus_1_itemsets: # iterate over keys in k_minus_1_itemsets, which are itemsets

            if itemset.issubset(reviews): # e.g. itemset = {10, 23} ; reviews = {10, 23, 33, 55}
                for other_reviewed_movie in ( reviews - itemset ): # other_reviewed_movie in {33 , 55}
                    current_superset = itemset | frozenset((other_reviewed_movie,)) 
                    if counted[current_superset]: continue
                    counted[current_superset] = True
                    # current_superset = {10, 23, 33} or {10, 23, 55}
                    counts[current_superset] += 1 # use itemset as key 
    return dict([(itemset, frequency) 
        for itemset, frequency in counts.items() if frequency >= min_support])



## Freq 1-item set

In [ ]:
num_favorable_by_movie.head()

,Favorable
MovieID,
1,66
2,5
3,4
4,21
5,6


In [ ]:
num_favorable_by_movie.columns, num_favorable_by_movie.head().index

(Index(['Favorable'], dtype='object'),
 Int64Index([1, 2, 3, 4, 5], dtype='int64', name='MovieID'))

In [ ]:
num_favorable_by_movie.head().reset_index()

,MovieID,Favorable
0,1,66
1,2,5
2,3,4
3,4,21
4,5,6


In [ ]:
? num_favorable_by_movie.iterrows

In [ ]:

frequent_itemsets = {}  # itemsets are sorted by length
min_support = 20

# k=1 candidates are the movies with more than min_support favourable reviews
frequent_itemsets[1] = dict((frozenset((movie_id,)), row["Favorable"])
                                for movie_id, row in num_favorable_by_movie.iterrows()
                                if row["Favorable"] > min_support)
print("There are {} movies with more than {} favorable reviews".format(len(frequent_itemsets[1]), min_support))

There are 162 movies with more than 20 favorable reviews


### Frequent k-itemsets

- #### <font color="red">`frequent_itemsets` is a **dictionary of dictionary**</font>
    - dictionary key is k (length of itemsets)
    - dictionary value is another dictionary which has key-value pairs like:
        - `frozenset({234, 456}) : 21`
        - where 234 and 456 are MovieID, 21 is the total count of users who rated this set as positive.

In [ ]:

from collections import defaultdict

# Recall that favorable_reviews_by_users is a dictionary with 
# user: {reviews} 
import sys

sys.stdout.flush() # Ensure printout still happens while programing running
for k in range(2, 20):
    # Generate candidates of length k, using the frequent itemsets of length k-1
    # Only store the frequent itemsets
    cur_frequent_itemsets = find_frequent_itemsets(favorable_reviews_by_users, 
                                                   frequent_itemsets[k-1],
                                                   min_support)
    if len(cur_frequent_itemsets) == 0:
        print("Did not find any frequent itemsets of length {}".format(k))
        sys.stdout.flush()
        break
    else:
        print("I found {} frequent itemsets of length {}".format(len(cur_frequent_itemsets), k))
        #print(cur_frequent_itemsets)
        sys.stdout.flush()
        frequent_itemsets[k] = cur_frequent_itemsets
# We aren't interested in the itemsets of length 1, so remove those
del frequent_itemsets[1]

I found 1242 frequent itemsets of length 2
I found 2214 frequent itemsets of length 3
I found 1767 frequent itemsets of length 4
I found 665 frequent itemsets of length 5
I found 96 frequent itemsets of length 6
I found 1 frequent itemsets of length 7
Did not find any frequent itemsets of length 8


In [ ]:
print(frequent_itemsets[6])
print(frequent_itemsets[7])

{frozenset({1, 50, 181, 172, 173, 174}): 20, frozenset({50, 98, 7, 56, 12, 174}): 20, frozenset({50, 172, 7, 56, 12, 174}): 20, frozenset({64, 50, 98, 7, 56, 174}): 21, frozenset({64, 50, 7, 56, 172, 174}): 20, frozenset({50, 98, 7, 56, 174, 79}): 21, frozenset({50, 100, 7, 56, 174, 79}): 20, frozenset({50, 7, 56, 172, 174, 79}): 22, frozenset({50, 181, 7, 56, 174, 79}): 20, frozenset({50, 7, 56, 89, 172, 174}): 21, frozenset({50, 98, 100, 7, 56, 174}): 20, frozenset({50, 98, 7, 56, 172, 174}): 21, frozenset({50, 98, 181, 7, 56, 174}): 20, frozenset({50, 100, 7, 56, 172, 174}): 21, frozenset({50, 7, 56, 172, 173, 174}): 21, frozenset({50, 181, 7, 56, 172, 174}): 24, frozenset({50, 210, 7, 56, 172, 174}): 22, frozenset({64, 50, 181, 7, 172, 174}): 20, frozenset({50, 98, 7, 172, 174, 79}): 20, frozenset({50, 98, 181, 7, 174, 79}): 20, frozenset({50, 181, 7, 172, 174, 79}): 20, frozenset({96, 50, 195, 7, 172, 174}): 20, frozenset({50, 98, 181, 7, 172, 174}): 20, frozenset({50, 181, 7, 172

In [ ]:
print("Found a total of {0} frequent itemsets".format(sum(len(itemsets) for itemsets in frequent_itemsets.values())))

Found a total of 5985 frequent itemsets


### Candidate rules

In [ ]:
# Now we create the association rules. First, they are candidates until the confidence has been tested
candidate_rules = []
import pdb
for itemset_length, itemset_counts in frequent_itemsets.items():
    
    pdb.set_trace()
    for itemset in itemset_counts.keys():
        for conclusion in itemset: #  {...} ==> conclusion
            premise = itemset - set((conclusion,))
            candidate_rules.append((premise, conclusion))
print("There are {} candidate rules".format(len(candidate_rules)))

> <ipython-input-57-bc583aec1af7>(7)<module>()
-> for itemset in itemset_counts.keys():
(Pdb) ?

Documented commands (type help <topic>):
EOF    c          d        h         list      q        rv       undisplay
a      cl         debug    help      ll        quit     s        unt      
alias  clear      disable  ignore    longlist  r        source   until    
args   commands   display  interact  n         restart  step     up       
b      condition  down     j         next      return   tbreak   w        
break  cont       enable   jump      p         retval   u        whatis   
bt     continue   exit     l         pp        run      unalias  where    

Miscellaneous help topics:
exec  pdb

(Pdb) l
  2  	candidate_rules = []
  3  	import pdb
  4  	for itemset_length, itemset_counts in frequent_itemsets.items():
  5  	
  6  	    pdb.set_trace()
  7  ->	    for itemset in itemset_counts.keys():
  8  	        for conclusion in itemset: #  {...} ==> conclusion
  9  	            premise =

In [ ]:
print(candidate_rules[:5])
print(candidate_rules[-5:])

[(frozenset({7}), 1), (frozenset({1}), 7), (frozenset({9}), 1), (frozenset({1}), 9), (frozenset({12}), 1)]
[(frozenset({64, 69, 172, 174, 50, 181}), 98), (frozenset({64, 98, 69, 172, 174, 50}), 181), (frozenset({64, 98, 172, 174, 50, 181}), 69), (frozenset({64, 98, 69, 174, 50, 181}), 172), (frozenset({64, 98, 69, 172, 50, 181}), 174)]


### Computing Confience for each rule

In [ ]:
# Now, we compute the confidence of each of these rules. 
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in favorable_reviews_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1
rule_confidence = {candidate_rule: 
        correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
              for candidate_rule in candidate_rules}

In [ ]:
# Choose only rules above a minimum confidence level
min_confidence = 0.9

In [ ]:
# Filter out the rules with poor confidence
rule_confidence = {rule: confidence for rule, confidence in rule_confidence.items() if confidence > min_confidence}
print(len(rule_confidence))

5113


In [ ]:
from operator import itemgetter
sorted_confidence = sorted(rule_confidence.items(), key=itemgetter(1), reverse=True)

In [ ]:
# note on itemgetter
print(itemgetter(1)([2,3]))
def myItemGetter(n):
    def f(items):
        return items[n]
    return f
print(myItemGetter(1)([2,3]))


3
3


In [ ]:
for index in range(0,3500,300):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise, conclusion))
    print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
    print("")

Rule #1
Rule: If a person recommends frozenset({89, 1}) they will also recommend 50
 - Confidence: 1.000

Rule #301
Rule: If a person recommends frozenset({50, 22, 98}) they will also recommend 174
 - Confidence: 1.000

Rule #601
Rule: If a person recommends frozenset({50, 173, 175}) they will also recommend 174
 - Confidence: 1.000

Rule #901
Rule: If a person recommends frozenset({96, 50, 69}) they will also recommend 174
 - Confidence: 1.000

Rule #1201
Rule: If a person recommends frozenset({56, 50, 172, 100}) they will also recommend 174
 - Confidence: 1.000

Rule #1501
Rule: If a person recommends frozenset({172, 204, 181, 174}) they will also recommend 50
 - Confidence: 1.000

Rule #1801
Rule: If a person recommends frozenset({172, 50, 181, 89, 222}) they will also recommend 174
 - Confidence: 1.000

Rule #2101
Rule: If a person recommends frozenset({64, 174, 183}) they will also recommend 50
 - Confidence: 0.960

Rule #2401
Rule: If a person recommends frozenset({176, 210, 50})

### Get movie titles

In [ ]:
# Even better, we can get the movie titles themselves from the dataset
movie_name_filename = os.path.join('.', "u.item")
movie_name_data = pd.read_csv(movie_name_filename, delimiter="|", header=None, encoding = "mac-roman")
movie_name_data.columns = ["MovieID", "Title", "Release Date", "Video Release", "IMDB", "<UNK>", "Action", "Adventure",
                           "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir",
                           "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]

In [ ]:
movie_name_data.head()

,MovieID,Title,Release Date,Video Release,IMDB,<UNK>,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [ ]:
def get_movie_name(movie_id):
    title_object = movie_name_data[movie_name_data["MovieID"] == movie_id]["Title"]
    title = title_object.values[0]
    return title

In [ ]:
get_movie_name(4)

'Get Shorty (1995)'

In [ ]:
for index in range(0, 3500, 300):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
    print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
    print("")

Rule #1
Rule: If a person recommends Blade Runner (1982), Toy Story (1995) they will also recommend Star Wars (1977)
 - Confidence: 1.000

Rule #301
Rule: If a person recommends Star Wars (1977), Braveheart (1995), Silence of the Lambs, The (1991) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 1.000

Rule #601
Rule: If a person recommends Star Wars (1977), Princess Bride, The (1987), Brazil (1985) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 1.000

Rule #901
Rule: If a person recommends Terminator 2: Judgment Day (1991), Star Wars (1977), Forrest Gump (1994) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 1.000

Rule #1201
Rule: If a person recommends Pulp Fiction (1994), Star Wars (1977), Empire Strikes Back, The (1980), Fargo (1996) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 1.000

Rule #1501
Rule: If a person recommends Empire Strikes Back, The (1980), Back to the Future (1985), Retu

### Evaluation

In [ ]:
# Evaluation using test data
test_dataset = all_ratings[~ (    all_ratings['UserID'].isin(range(200)) ) 
    ]
test_favorable = test_dataset[test_dataset["Favorable"]]
#test_not_favourable = test_dataset[~test_dataset["Favourable"]]
test_favorable_by_users = dict((k, frozenset(v.values)) for k, v in test_favorable.groupby("UserID")["MovieID"])
#test_not_favourable_by_users = dict((k, frozenset(v.values)) for k, v in test_not_favourable.groupby("UserID")["MovieID"])
#test_users = test_dataset["UserID"].unique()

In [ ]:
test_dataset[:5]

,UserID,MovieID,Rating,Datetime,Favorable
3,244,51,2,1997-11-27 05:02:03,False
5,298,474,4,1998-01-07 14:20:06,True
7,253,465,5,1998-04-03 18:34:27,True
8,305,451,3,1998-02-01 09:20:17,False
11,286,1014,5,1997-11-17 15:38:45,True


In [ ]:
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in test_favorable_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1

In [ ]:
test_confidence = {candidate_rule: correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
                   for candidate_rule in rule_confidence}
print(len(test_confidence))

5113


In [ ]:
sorted_test_confidence = sorted(test_confidence.items(), key=itemgetter(1), reverse=True)
print(sorted_test_confidence[:5])

[((frozenset({194, 181}), 50), 1.0), ((frozenset({1, 173, 222}), 50), 1.0), ((frozenset({56, 173, 181}), 50), 1.0), ((frozenset({56, 181, 183}), 50), 1.0), ((frozenset({64, 172, 196}), 50), 1.0)]


In [ ]:
for index in range(0, 3500, 300):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
    print(" - Train Confidence: {0:.3f}".format(rule_confidence.get((premise, conclusion), -1)))
    print(" - Test Confidence: {0:.3f}".format(test_confidence.get((premise, conclusion), -1)))
    print("")

Rule #1
Rule: If a person recommends Blade Runner (1982), Toy Story (1995) they will also recommend Star Wars (1977)
 - Train Confidence: 1.000
 - Test Confidence: 0.915

Rule #301
Rule: If a person recommends Star Wars (1977), Braveheart (1995), Silence of the Lambs, The (1991) they will also recommend Raiders of the Lost Ark (1981)
 - Train Confidence: 1.000
 - Test Confidence: 0.911

Rule #601
Rule: If a person recommends Star Wars (1977), Princess Bride, The (1987), Brazil (1985) they will also recommend Raiders of the Lost Ark (1981)
 - Train Confidence: 1.000
 - Test Confidence: 0.863

Rule #901
Rule: If a person recommends Terminator 2: Judgment Day (1991), Star Wars (1977), Forrest Gump (1994) they will also recommend Raiders of the Lost Ark (1981)
 - Train Confidence: 1.000
 - Test Confidence: 0.919

Rule #1201
Rule: If a person recommends Pulp Fiction (1994), Star Wars (1977), Empire Strikes Back, The (1980), Fargo (1996) they will also recommend Raiders of the Lost Ark (1981

## Using package `efficient_apriori` for association rules mining

In [ ]:
!pip install  efficient_apriori  

- See https://efficient-apriori.readthedocs.io/en/latest/
> for working examples and note what to do with large files.

### Recall that `favorable_reviews_by_users` is a dictionary 

- with key-value pairs being:
    - UserID as a key 
    - a set of favorately reviewed movies' IDs as a value

In [ ]:

transactions = favorable_reviews_by_users.values()
len(transactions)

199

In [ ]:
import math 
from efficient_apriori import apriori
itemsets, rules = apriori(transactions, min_support=0.1, min_confidence=.9)
for r in rules[0: 3500: 300]:
    print ("efficient_apriori says: ", r)
    conf = rule_confidence[(frozenset(r.lhs), (r.rhs)[0])]
    print("Our program says, confidence={:0.3f}".format(conf), 
          math.isclose(conf, r.confidence) )

efficient_apriori says:  {172} -> {50} (conf: 0.915, supp: 0.271, lift: 1.821, conv: 5.870)
Our program says, confidence=0.915 True
efficient_apriori says:  {172, 195} -> {50} (conf: 0.943, supp: 0.166, lift: 1.876, conv: 8.706)
Our program says, confidence=0.943 True
efficient_apriori says:  {135, 195} -> {174} (conf: 0.957, supp: 0.111, lift: 2.572, conv: 14.447)
Our program says, confidence=0.957 True
efficient_apriori says:  {1, 50, 96} -> {174} (conf: 1.000, supp: 0.101, lift: 2.689, conv: 628140703.518)
Our program says, confidence=1.000 True
efficient_apriori says:  {7, 98, 176} -> {174} (conf: 0.955, supp: 0.106, lift: 2.567, conv: 13.819)
Our program says, confidence=0.955 True
efficient_apriori says:  {22, 172, 195} -> {174} (conf: 1.000, supp: 0.111, lift: 2.689, conv: 628140703.518)
Our program says, confidence=1.000 True
efficient_apriori says:  {50, 79, 258} -> {174} (conf: 0.952, supp: 0.101, lift: 2.561, conv: 13.191)
Our program says, confidence=0.952 True
efficient_ap

## Some terms in Association Rules Mining


 Rule LHS and RHS: 
$$
 antecedent \longrightarrow consequent 
$$

$$
\begin{align}
conf(X\longrightarrow Y) &= supp(X\cup Y)/supp(X) \\
&= Prob(X\ and\ Y)/Prob(X) \\
&= Prob(Y | X)
\end{align}
$$
> Confidence: <font size="-1">The confidence of a rule X->Y is the probability of seeing the consequent in a transaction given that it also contains the antecedent. Note that the metric is not symmetric or directed; for instance, the confidence for X->Y is different than the confidence for Y->X. The confidence is 1 (maximal) for a rule X->Y if the consequent and antecedent always occur together. A problem with confidence is that it is sensitive to the frequency of the consequent (Y) in the database. Caused by the way confidence is calculated, consequents with higher support will automatically produce higher confidence values even if there exists no association between the items.</font>
$$
\begin{align}
 Lift(X\rightarrow Y)&=\frac{supp(X\cup Y)}{supp(X)*supp(Y)} \\
&=\frac{Confidence(X\rightarrow Y)}{supp(Y)} \\
&=\frac{Prob(X\  and\   Y)}{Prob(X)*Prob(Y)} 
\end{align}
$$


> Lift <font size="-1">measures how many times more often X and Y occur together than expected if they were statistically independent. If items are independent, the lift is 1.</font>

$$
\begin{align}
Conviction(X\rightarrow Y) &=\frac{1-supp(Y)}{1-conf(X\rightarrow Y)} \\
&= P(X)P(not\ Y)/P(X\ and\ not\ Y)
\end{align}
$$

> Conviction <font size="-1">compares the probability that X appears without Y if they were dependent with the actual frequency of the appearance of X without Y. A high conviction value means that the consequent is highly depending on the antecedent. For instance, in the case of a perfect confidence score, the denominator becomes 0 (due to 1 - 1) for which the conviction score is defined as 'inf'. Similar to lift, if items are independent, the conviction is 1. In that respect it is similar to lift, however, it contrast to lift it is a directed measure since it also uses the information of the absence of the consequent. An interesting fact is that conviction is monotone in confidence and lift.</font>

$${levarage}(A\rightarrow C) = {support}(A\rightarrow C) - {support}(A) * {support}(C), \;\;\; \text{range: } [-1, 1]$$


> Leverage <font size="-1">computes the difference between the observed frequency of A and C appearing together and the frequency that would be expected if A and C were independent. An leverage value of 0 indicates independence.</font>

## Package mlxtend

In [ ]:
!pip install -U mlxtend

     |████████████████████████████████| 1.4MB 5.4MB/s 
  Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth


dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


In [ ]:
frequent_itemsets = fpgrowth(df, min_support=0.6, use_colnames=True) # faster than apriori
### alternatively:
#frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
#frequent_itemsets = fpmax(df, min_support=0.6, use_colnames=True)

frequent_itemsets

,support,itemsets
0,1.0,(Kidney Beans)
1,0.8,(Eggs)
2,0.6,(Yogurt)
3,0.6,(Onion)
4,0.6,(Milk)
5,0.8,"(Eggs, Kidney Beans)"
6,0.6,"(Kidney Beans, Yogurt)"
7,0.6,"(Eggs, Onion)"
8,0.6,"(Kidney Beans, Onion)"
9,0.6,"(Eggs, Kidney Beans, Onion)"


In [ ]:
# Rule gneration
from mlxtend.frequent_patterns import association_rules

association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Eggs),(Kidney Beans),0.8,1.0,0.8,1.00,1.00,0.00,inf
1,(Kidney Beans),(Eggs),1.0,0.8,0.8,0.80,1.00,0.00,1.0
2,(Yogurt),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf
3,(Eggs),(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6
4,(Onion),(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf
5,(Onion),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf
6,"(Eggs, Kidney Beans)",(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6
7,"(Eggs, Onion)",(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf
8,"(Kidney Beans, Onion)",(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf
9,(Eggs),"(Kidney Beans, Onion)",0.8,0.6,0.6,0.75,1.25,0.12,1.6



- FP-Growth is an frequent pattern mining algorithm that does not require candidate generation. Internally, it uses a so-called FP-tree (frequent pattern tree) datastrucure without generating the candidate sets explicitely, which makes is particularly attractive for large datasets.

- **Reference**
Han, Jiawei, Jian Pei, Yiwen Yin, and Runying Mao. "Mining frequent patterns without candidate generation. "A frequent-pattern tree approach." Data mining and knowledge discovery 8, no. 1 (2004): 53-87.

- More info: http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/#example-1-generating-association-rules-from-frequent-itemsets